<a href="https://colab.research.google.com/github/Ahmedabdelwaly/NlpProject/blob/main/Copy_of_Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets arabert bert_score evaluate rouge_score

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.0/185.0 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast, DataCollatorForLanguageModeling, TrainingArguments, Trainer
from arabert.preprocess import ArabertPreprocessor
from datasets import load_dataset
import torch

In [ ]:
# Load 1% of OSCAR Arabic dataset
from datasets import load_dataset

dataset = load_dataset(
    "oscar-corpus/oscar",
    "unshuffled_deduplicated_ar",
    split="train[:10000]",
    cache_dir="/content"
)

# Train/Test split 90/10
dataset = dataset.train_test_split(test_size=0.1)

# Check dataset size
print(dataset)


Generating train split:   0%|          | 0/9006977 [00:00<?, ? examples/s]

In [ ]:
model_name = "aubmindlab/aragpt2-base"
tokenizer = GPT2TokenizerFast.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

In [ ]:
# Load tokenizer & model
model_name = "aubmindlab/aragpt2-base"
tokenizer = GPT2TokenizerFast.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = GPT2LMHeadModel.from_pretrained(model_name)

# Arabert Preprocessor
arabert_prep = ArabertPreprocessor(model_name=model_name)

# Preprocess texts
def preprocess_function(example):
    text = arabert_prep.preprocess(example["text"])
    return {"text": text}

#dataset = dataset.map(preprocess_function)

In [ ]:
# Save train and test to text files (for TextDataset)
train_texts = "\n".join(dataset['train']['text'])
test_texts = "\n".join(dataset['test']['text'])

with open('train.txt', 'w', encoding='utf-8') as f:
    f.write(train_texts)

with open('test.txt', 'w', encoding='utf-8') as f:
    f.write(test_texts)


In [ ]:
from transformers import TextDataset

# Load datasets
train_dataset = TextDataset(
    tokenizer=tokenizer, file_path='train.txt', block_size=128
)
test_dataset = TextDataset(
    tokenizer=tokenizer, file_path='test.txt', block_size=128
)

# Data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Training arguments
training_args = TrainingArguments(
    output_dir='./output',
    num_train_epochs=3,
    per_device_train_batch_size=32,
    learning_rate=5e-5,
    save_steps=500,
    logging_steps=10,
    report_to="none",
    fp16=True
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

# Train
trainer.train()


/usr/local/lib/python3.11/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
10,8.210500
20,6.700800
30,6.344400
40,6.165600
50,5.872900
60,5.865300
70,5.730000
80,5.570000
90,5.567700
100,5.600900


TrainOutput(global_step=3681, training_loss=4.809786138894667, metrics={'train_runtime': 1661.9389, 'train_samples_per_second': 70.871, 'train_steps_per_second': 2.215, 'total_flos': 7693939851264000.0, 'train_loss': 4.809786138894667, 'epoch': 3.0})

In [ ]:
model.save_pretrained("./arabic_autocomplete_model")
tokenizer.save_pretrained("./arabic_autocomplete_model")

('./arabic_autocomplete_model/tokenizer_config.json',
 './arabic_autocomplete_model/special_tokens_map.json',
 './arabic_autocomplete_model/vocab.json',
 './arabic_autocomplete_model/merges.txt',
 './arabic_autocomplete_model/added_tokens.json',
 './arabic_autocomplete_model/tokenizer.json')

In [ ]:
!zip -r arabic_model.zip /content/arabic_autocomplete_model
from google.colab import files
files.download("arabic_model.zip")

  adding: content/arabic_autocomplete_model/ (stored 0%)
  adding: content/arabic_autocomplete_model/generation_config.json (deflated 23%)
  adding: content/arabic_autocomplete_model/special_tokens_map.json (deflated 60%)
  adding: content/arabic_autocomplete_model/merges.txt (deflated 77%)
  adding: content/arabic_autocomplete_model/vocab.json (deflated 75%)
  adding: content/arabic_autocomplete_model/tokenizer_config.json (deflated 74%)
  adding: content/arabic_autocomplete_model/model.safetensors (deflated 7%)
  adding: content/arabic_autocomplete_model/config.json (deflated 52%)
  adding: content/arabic_autocomplete_model/tokenizer.json (deflated 85%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
def predict(text, k=5):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    text = arabert_prep.preprocess(text)
    indexed_tokens = tokenizer.encode(text, return_tensors="pt").to(device)

    # Move model to device
    model.to(device)
    model.eval()

    with torch.no_grad():
        outputs = model(indexed_tokens)
        soft_max = torch.nn.Softmax(dim=0)
        probs = soft_max(outputs[0][0, -1, :])

        sorted_probs, indices = torch.topk(probs, k)
        preds = [index.item() for index in indices]

    predicted_words = tokenizer.decode(preds).split()

    return predicted_words


# Test prediction
predict("قررت المحكمة")



['الابتدائية', 'العليا', 'العسكرية', 'في', 'الجنائية']

In [ ]:
predict("استعد الرئيس")

In [ ]:
def top_k_accuracy(test_data, k=5):
    correct = 0

    for prompt, true_word in test_data:
        text = arabert_prep.preprocess(prompt)
        input_ids = tokenizer.encode(text, return_tensors="pt").to(device)

        with torch.no_grad():
            outputs = model(input_ids)
            next_token_logits = outputs.logits[0, -1, :]
            probs = F.softmax(next_token_logits, dim=0)
            topk = torch.topk(probs, k).indices.tolist()

        decoded_topk = [tokenizer.decode([token]).strip() for token in topk]

        if true_word in decoded_topk:
            correct += 1

        print(f"Prompt: {prompt}\nExpected: {true_word}\nTop-{k}: {decoded_topk}\n")

    accuracy = correct / len(test_data)
    print(f"Top-{k} Accuracy: {accuracy:.2f}")
    return accuracy


In [ ]:
def build_test_data_from_textfile(filepath, min_len=3):
    test_data = []
    with open(filepath, "r", encoding="utf-8") as f:
        for line in f:
            words = line.strip().split()
            if len(words) >= min_len:
                prompt = " ".join(words[:2])
                next_word = words[2]
                test_data.append((prompt, next_word))
    return test_data

# Load test.txt into test_data
test_data = build_test_data_from_textfile("test.txt")


In [ ]:
def compute_perplexity(sentence):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    inputs = tokenizer(arabert_prep.preprocess(sentence), return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs, labels=inputs["input_ids"])
        loss = outputs.loss
    return torch.exp(loss).item()

total_perplexity = 0
for prompt, _ in test_data:
    total_perplexity += compute_perplexity(prompt)

avg_perplexity = total_perplexity / len(test_data)
print(f"🔍 Average Perplexity on Test Set: {avg_perplexity:.2f}")


🔍 Average Perplexity on Test Set: 10754.83


In [ ]:
from evaluate import load
bertscore = load("bertscore")

prompts = [pair[0] for pair in test_data]
completions = [predict(prompt)[0] for prompt in prompts]

bert_result = bertscore.compute(predictions=completions, references=prompts, lang="ar")
print(f"BERTScore (F1): {sum(bert_result['f1']) / len(bert_result['f1']):.4f}")


BERTScore (F1): 0.7038


In [ ]:
from evaluate import load

# Load the BERTScore metric
bertscore = load("bertscore")

# Your updated test examples
prompts = ["قررت المحكمة", "استعد الرئيس"]
completions = ["الجنائية", "الامريكي"]

# Compute BERTScore
bert_result = bertscore.compute(predictions=completions, references=prompts, lang="ar")

# Print detailed scores per sentence
print("🔎 Detailed BERTScore (F1) per example:\n")
for i in range(len(prompts)):
    print(f"Prompt     : {prompts[i]}")
    print(f"Prediction : {completions[i]}")
    print(f"BERTScore F1: {bert_result['f1'][i]:.4f}")
    print("-" * 40)

# Print average score
avg_score = sum(bert_result['f1']) / len(bert_result['f1'])
print(f"✅ Average BERTScore (F1): {avg_score:.4f}")

🔎 Detailed BERTScore (F1) per example:

Prompt     : قررت المحكمة
Prediction : الجنائية
BERTScore F1: 0.7096
----------------------------------------
Prompt     : استعد الرئيس
Prediction : الامريكي
BERTScore F1: 0.6894
----------------------------------------
✅ Average BERTScore (F1): 0.6995


In [ ]:
from evaluate import load
import torch

# Load the BERTScore metric
bertscore = load("bertscore")

# Define test examples
prompts = ["قررت المحكمة", "استعد الرئيس"]
completions = ["الجنائية", "الامريكي"]

# Compute BERTScore
bert_result = bertscore.compute(predictions=completions, references=prompts, lang="ar")

# 🔢 Perplexity function
def compute_perplexity(prompt):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    input_ids = tokenizer(arabert_prep.preprocess(prompt), return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**input_ids, labels=input_ids["input_ids"])
        loss = outputs.loss
    return torch.exp(loss).item()

# 🔎 Print detailed results
print("🔎 BERTScore + Perplexity Evaluation:\n")
total_perplexity = 0

for i in range(len(prompts)):
    perplexity = compute_perplexity(prompts[i])
    total_perplexity += perplexity

    print(f"Prompt       : {prompts[i]}")
    print(f"Prediction   : {completions[i]}")
    print(f"BERTScore F1 : {bert_result['f1'][i]:.4f}")
    print(f"Perplexity   : {perplexity:.2f}")
    print("-" * 40)

# ✅ Averages
avg_f1 = sum(bert_result['f1']) / len(bert_result['f1'])
avg_ppl = total_perplexity / len(prompts)

print(f"✅ Average BERTScore (F1): {avg_f1:.4f}")
print(f"✅ Average Perplexity   : {avg_ppl:.2f}")


🔎 BERTScore + Perplexity Evaluation:

Prompt       : قررت المحكمة
Prediction   : الجنائية
BERTScore F1 : 0.7096
Perplexity   : 342.08
----------------------------------------
Prompt       : استعد الرئيس
Prediction   : الامريكي
BERTScore F1 : 0.6894
Perplexity   : 1152.15
----------------------------------------
✅ Average BERTScore (F1): 0.6995
✅ Average Perplexity   : 747.12
